# Pre Process Pipeline

## ML

In [27]:
import os
import librosa
import numpy as np
import soundfile as sf
import scipy.signal
import noisereduce as nr

In [28]:
# === CONFIGURATION ===
MAIN_FOLDER_PATH = ".."
DATASET_PATH = os.path.join(MAIN_FOLDER_PATH, "DataSet")
OUTPUT_FOLDER = os.path.join(MAIN_FOLDER_PATH, "PreProcessedDataSet_for_ML")
TARGET_SR = 16000   # sapmle rate
SILENCE_TOP_DB = 40
HIGHPASS_CUTOFF = 60  # cutoff to remove low-frequency noise

# Ensure output directory exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

### Used Functions For Pre-processing Explained

The reason for using these specific functinsfor our pre-processing pipeline for *traditional ML speech emotion recognition* on the **CREMA-D dataset**:

---

#### **`load_and_resample_audio(file_path, sr=TARGET_SR)`**  
**Reason for using:**  
- Ensures all audio files have the same sampling rate (e.g., 16kHz).  
- Different recordings might have varying sample rates, which makes feature extraction (like MFCC, Chroma, etc.) inconsistent.  
- Having a consistent `sr` allows uniform time and frequency resolution for all extracted features.  

*In short: Without uniform sampling rate, features would not be comparable.*

---

#### **`convert_to_mono(audio)`**  
**Reason for using:**  
- CREMA-D clips can be stereo.  
- Traditional audio feature extraction (MFCC, Chroma, Spectral Contrast) works on mono signals — stereo channels can cause duplication and confusion.  
- Mono conversion simplifies the signal and avoids unnecessary complexity.  

*Mono ensures one clean signal for consistent, meaningful feature extraction.*

---

#### **`trim_silence(audio, top_db=SILENCE_TOP_DB)`**  
**Reason for using:**  
- Light silence trimming (e.g., `top_db=40`) removes long, unnecessary silences at the beginning and end.  
- Long silences would inflate feature lengths or distort statistics (for features like zero-crossing rate or spectral entropy).  
- But trimming is gentle, so we don’t remove emotional pauses inside the speech.  

*Helps keep the audio focused on speech content, improving feature quality.*

---

#### **`normalize_audio(audio)`**  
**Reason for using:**  
- Normalizes audio so that its peak amplitude is 1.  
- Feature extractors are amplitude-sensitive (especially energy-based features).  
- Without normalization, two identical speech samples at different recording volumes could look like different emotional intensities.  
- Peak normalization maintains relative loudness dynamics but makes the scale consistent.  

*Eliminates unwanted loudness variations while preserving natural expressive dynamics.*

---

#### **`apply_highpass_filter(audio, sr, cutoff=HIGHPASS_CUTOFF)`**  
**Reason for using:**  
- Removes low-frequency hums and rumble below 80Hz (e.g., air conditioning noise, mic stand vibrations).  
- These low frequencies are *not useful* for emotion; emotion features live in mid- and high-frequency ranges (pitch, formants, harmonics).  
- Prevents “pollution” of features like MFCC or spectral contrast by meaningless low-frequency noise.  

*Keeps the frequency content clean and relevant for speech emotion features.*


In [29]:
# =================== PREPROCESSING FUNCTIONS ===================

def load_and_resample_audio(file_path, sr=TARGET_SR):
    """Load audio file and resample."""
    audio, sr = librosa.load(file_path, sr=sr)
    return audio, sr

def convert_to_mono(audio):
    """Convert stereo audio to mono."""
    return librosa.to_mono(audio)

def trim_silence(audio, top_db=SILENCE_TOP_DB):
    """Trim leading and trailing silence (if any)."""
    trimmed_audio, _ = librosa.effects.trim(audio, top_db=top_db)
    return trimmed_audio

def normalize_audio(audio):
    """Normalize audio amplitude to peak value of 1."""
    return audio / np.max(np.abs(audio))

def apply_highpass_filter(audio, sr, cutoff=HIGHPASS_CUTOFF):
    """Apply high-pass filter to remove low-frequency noise."""
    sos = scipy.signal.butter(10, cutoff, btype='highpass', fs=sr, output='sos')
    return scipy.signal.sosfilt(sos, audio)

def is_not_empty(audio):
    return np.max(np.abs(audio))



### Functions We Could Have Used But Avoided and Why

The below functions that you will see on the next python cell are the function that we could've used but decided not to use for the following reasons:

#### **Noise Reduction**  

**Why we avoided it:**  
- CREMA-D is a *studio-recorded* dataset with relatively clean audio;  
- Noise reduction algorithms can sometimes distort the natural tone of speech, which is critical for emotion recognition;  
- Risk of over-smoothing, harming features like pitch variance or spectral dynamics that are crucial for emotion detection.

---

#### **Extreme Silence Removal (aggressive trimming)**  

**Why we avoided it:**  
- Too aggressive silence removal could cut natural pauses that carry emotional cues (hesitation, sighs, or dramatic pauses).  
- Emotion detection benefits from subtle prosody and timing variations.  
- We stuck to `top_db=40` light trimming instead.

---

#### **Volume Standardization (RMS normalization)**  

**Why avoided:**  
- RMS normalization can distort amplitude-based features like loudness, which are *strong indicators* of emotion (e.g., anger vs. calm).  
- Peak normalization was chosen to preserve relative dynamics.

---

#### **Band-Pass Filtering (instead of high-pass)**  

**Why avoided:**  
- CREMA-D recordings already have controlled frequency ranges.  
- Over-filtering could remove important frequency components (high frequencies carry stress, low frequencies carry warmth).  
- High-pass filter (80Hz) was enough to clean up low rumble without cutting expressive content.

---

#### **Time-Stretch or Pitch-Shifting (Data Augmentation)**  

These are great for training large neural models but **not suitable** at preprocessing for ML feature extraction.  
- Feature extraction should happen on clean, original audio.  
- Augmentation could introduce bias or misrepresent real emotional cues.


---




In [30]:
# =================== PREPROCESSING FUNCTIONS WE DIDN'T USE ===================

def reduce_noise(audio, sr):
    """Perform noise reduction using spectral gating."""
    return nr.reduce_noise(y=audio, sr=sr)

def aggressive_trim(audio, top_db=20):
    """Remove even quieter silences more aggressively."""
    trimmed_audio, _ = librosa.effects.trim(audio, top_db=top_db)
    return trimmed_audio

def rms_normalize(audio):
    """Normalize based on RMS energy."""
    rms = np.sqrt(np.mean(audio**2))
    return audio / rms

def bandpass_filter(audio, sr, lowcut=80, highcut=8000):
    """Apply band-pass filter."""
    sos = scipy.signal.butter(10, [lowcut, highcut], btype='band', fs=sr, output='sos')
    return scipy.signal.sosfilt(sos, audio)


The following function is the pipeline process. It gets the audio file does all the processing functions we chose that where suitable and important to do and after processing the audio file the file will be saved as a new file in another foler where all the preProcessed .wav files are there.

In [ ]:
# =================== PIPELINE PROCESS FUNCTION ===================

def process_audio_file(file_path):
    """Apply full preprocessing pipeline to a single audio file (for traditional ML)."""
    try:
        #print(f"Processing: {file_path}")
        audio, sr = load_and_resample_audio(file_path)
        if(is_not_empty(audio)):
            audio = convert_to_mono(audio)
            audio = trim_silence(audio)
            audio = normalize_audio(audio)
            audio = apply_highpass_filter(audio, sr)

            # Note: No padding or truncation — audio length can vary for ML models
            output_file = os.path.join(OUTPUT_FOLDER, os.path.basename(file_path))
            sf.write(output_file, audio, sr)
            #print(f"Saved: {output_file}")

    except Exception as error:
        print(f"Error processing {file_path}: {error}")


In [32]:
# =================== MAIN PIPELINE ===================

def preprocess_dataset():
    """Process all audio files in the dataset (traditional ML pipeline)."""
    for filename in os.listdir(DATASET_PATH):
        if filename.lower().endswith(".wav"):
            file_path = os.path.join(DATASET_PATH, filename)
            process_audio_file(file_path)
    print("\nAll audio files have been preprocessed and saved in: ", OUTPUT_FOLDER)

In [33]:
# =================== EXECUTION ===================
preprocess_dataset()

deletedddddddddddddddddddddddddddddddddddddddddddddddddddd

All audio files have been preprocessed and saved in:  ..\PreProcessedDataSet_for_ML
